## **1. Research Questions introduction**



	“Quais são os fatores (género, número de ratings, e popularidade) que mais influenciam a média de avaliação dos filmes no MovieLens?”

Isto permite:
	•	Explorar a estrutura e a distribuição dos dados (EDA);
	•	Criar métricas derivadas (popularidade, número de votos, média ponderada);
	•	Usar modelos simples (regressão linear / árvore de decisão) para quantificar o impacto de cada variável;
	•	E escalar a análise — localmente em DuckDB, e facilmente replicável em cloud (Athena/BigQuery).

## **2. Pipeline**

### **2.1. Import necessary libraries and packages**

In [2]:
import polars as pl
import pandas as pd
import sys
import os

In [3]:
print(sys.executable)

#"C:\Users\SaraEstevesHenriques\AppData\Local\Programs\Python\Python313\python.exe" -m pip install polars

C:\Users\RitaML\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe


In [4]:
print ("This is the file directory:", os.getcwd())
parquet_path = os.path.join(os.getcwd(), "moviedetails.parquet")

print("Python executable:", sys.executable)
print("Parquet exists:", os.path.exists(parquet_path))


This is the file directory: c:\Users\RitaML\Documents\GitHub\BDF25_7\BDF25_7_data\big_data\ml-32m
Python executable: C:\Users\RitaML\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe
Parquet exists: True


### **2.1. Import dataset**

#### **2.1.1. Import parquet - small ratings version!!**

In [7]:
# Eager read (loads into memory)
try:
    df_small = pl.read_parquet(parquet_path)
    print("Eager read shape:", df_small.shape)
    display(df_small.head())
except Exception as e:
    print("Eager read failed:", e)

Eager read shape: (9255, 6)


movieId,title,genres,average_rating,rating_count,tag_count_per_movie
i64,str,str,f64,i64,i64
161582,"""Hell or High Water (2016)""","""Crime|Drama""",3.5625,3176,3176
1982,"""Halloween (1978)""","""Horror""",3.722222,8586,8586
52245,"""Blades of Glory (2007)""","""Comedy|Romance""",3.088235,2482,2482
62,"""Mr. Holland's Opus (1995)""","""Drama""",3.70625,6800,6800
508,"""Philadelphia (1993)""","""Drama""",3.613636,27324,27324


#### **2.1.2. Import cvs - results obtained and exported from query in AWS Athena**

In [8]:
#the cvs is on this folder and its called moviedetails csv
df = pl.read_csv("moviedetails.csv")
print("CSV read successfully. Shape:", df.shape)
display(df.head())


CSV read successfully. Shape: (87585, 7)


movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie
i64,str,str,f64,i64,i64,i64
20,"""Money Train (1995)""","""Action|Comedy|Crime|Drama|Thri…",2.871458,589922,589922,4306
171,"""Jeffrey (1995)""","""Comedy|Drama""",3.593487,27132,27132,1428
315,"""Specialist, The (1994)""","""Action|Drama|Thriller""",2.893674,102717,102717,11413
331,"""Tom & Viv (1994)""","""Drama""",3.289855,1656,1656,552
386,"""S.F.W. (1994)""","""Drama""",2.822281,5655,5655,377


### **2.2. Data Preparation**

2.	Calcular:

- nº total de utilizadores (200948), filmes (87585) e avaliações (32M)
- distribuição de rating (média, mediana, desvio padrão)
- nº de filmes por género
- nº médio de ratings por filme e por utilizador

3.	Visualizar:
	
- histograma das classificações
- top 10 géneros mais avaliados
- relação entre nº de ratings e média por filme


#### 2.1.1. Titles analysis

The titles in most cases have the actual title and the movie year. The first step is to split both of this informations. 

In [9]:
df.head(20)

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie
i64,str,str,f64,i64,i64,i64
20,"""Money Train (1995)""","""Action|Comedy|Crime|Drama|Thri…",2.871458,589922,589922,4306
171,"""Jeffrey (1995)""","""Comedy|Drama""",3.593487,27132,27132,1428
315,"""Specialist, The (1994)""","""Action|Drama|Thriller""",2.893674,102717,102717,11413
331,"""Tom & Viv (1994)""","""Drama""",3.289855,1656,1656,552
386,"""S.F.W. (1994)""","""Drama""",2.822281,5655,5655,377
…,…,…,…,…,…,…
1233,"""Boot, Das (Boat, The) (1981)""","""Action|Drama|War""",4.129334,14334228,14334228,15804
1545,"""Ponette (1996)""","""Drama""",3.849922,7716,7716,643
1772,"""Blues Brothers 2000 (1998)""","""Action|Comedy|Musical""",2.544862,135149,135149,3143


In [10]:
# Extract year and remove it from title
df = df.with_columns([
    # Extract year, the 4 numbers inside the "". If no year is found it will be null
    pl.col("title").str.extract(r'\((\d{4})\)', 1).cast(pl.Int64).alias("year"),
    # Remove year from title (removes the pattern " (YYYY)" or "(YYYY)")
    pl.col("title").str.replace(r'\s*\(\d{4}\)', '').str.strip_chars('"').alias("title")
])

In [11]:
df.head()

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie,year
i64,str,str,f64,i64,i64,i64,i64
20,"""Money Train""","""Action|Comedy|Crime|Drama|Thri…",2.871458,589922,589922,4306,1995
171,"""Jeffrey""","""Comedy|Drama""",3.593487,27132,27132,1428,1995
315,"""Specialist, The""","""Action|Drama|Thriller""",2.893674,102717,102717,11413,1994
331,"""Tom & Viv""","""Drama""",3.289855,1656,1656,552,1994
386,"""S.F.W.""","""Drama""",2.822281,5655,5655,377,1994


The following line allows to conclude that not every movie has the correspondent year set.

In [ ]:
df.null_count()
#615 missing values on the year, 3153 on average_rating

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie,year
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,3153,0,0,0,615


In [13]:
df_noyear = df.filter(pl.col("year").is_null())
df_noyear.head ()

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie,year
i64,str,str,f64,i64,i64,i64,i64
146796,"""Le strelle nel fosso""","""(no genres listed)""",null,0,1,0,null
156605,"""Paterson""","""(no genres listed)""",3.712108,177940,177940,1148,null
191403,"""New Neighbors, Old Fights""","""(no genres listed)""",3.5,1,0,1,null
195217,"""Star Wars: Dresca""","""Sci-Fi""",2.590909,88,88,11,null
205074,"""Falling Inn Love""","""Comedy|Romance""",2.7,735,735,105,null


For the purpose of our analysis i think we'll need to remove the 3153 movies without average rating, these are movies without any rating

Rita: Agreed!

In [14]:
df_norating = df.filter(pl.col("average_rating").is_null())
df_norating.head (10)

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie,year
i64,str,str,f64,i64,i64,i64,i64
101237,"""9500 Liberty""","""Documentary""",null,0,4,0,2009
109647,"""Bengazi""","""Adventure|Crime|Drama""",null,0,4,0,1955
110822,"""Super Fly T.N.T.""","""Action|Crime|Drama""",null,0,1,0,1973
116594,"""Driftin' River""","""Western""",null,0,1,0,1946
121395,"""Mother is a Freshman""","""Comedy""",null,0,3,0,1949
123794,"""Killer by Night""","""Crime|Thriller""",null,0,3,0,1972
129988,"""Tiffany Memorandum""","""Action|Romance""",null,0,1,0,1967
133497,"""Goldface il fantastico Superma…","""Adventure|Crime""",null,0,1,0,1967
138658,"""I Put a Hit On You""","""Comedy|Romance|Thriller""",null,0,1,0,2014


#### 2.1.2. Genres

There are movies with multiple genres and others with no gender indicated "(no genres listed)". The first action here is to treat the no genres listed as nulls. One possible approach to split the genders is use explode funcion, creating a line per movie per gender.

In [15]:
#Tranform no (no genres list) to null values
df = df.with_columns(
    pl.when(pl.col("genres").str.to_lowercase().str.strip_chars() == "(no genres listed)")
    .then(None)
    .otherwise(pl.col("genres"))
    .alias("genres")
    )

In [16]:
df_nogenre = df.filter(pl.col("genres").is_null())
df_nogenre.head (10)

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie,year
i64,str,str,f64,i64,i64,i64,i64
123989,"""Spike of Bensonhurst""",null,4.5,26,26,2,1988
128393,"""Highway Racer""",null,3.5,21,21,3,1977
131320,"""Fort Yuma""",null,3.5,1,1,1,1955
132896,"""Pelota""",null,3.5,2,2,2,1983
137140,"""Blue Ridge""",null,5.0,1,0,1,2012
139375,"""Après la bataille""",null,null,0,1,0,2014
142128,"""The Missing Corpse""",null,1.5,1,0,1,1945
143217,"""Middle Age Crazy""",null,3.5,2,2,2,1980
144998,"""Rebels in Canada""",null,null,0,1,0,1965


In [17]:

df.null_count()


movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie,year
u32,u32,u32,u32,u32,u32,u32,u32
0,0,7080,3153,0,0,0,615


7080 movies do not have genres associated.

In [18]:
df_exploded = df.with_columns(
    pl.col("genres").str.split("|")
).explode("genres")

df_exploded.head()

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie,year
i64,str,str,f64,i64,i64,i64,i64
20,"""Money Train""","""Action""",2.871458,589922,589922,4306,1995
20,"""Money Train""","""Comedy""",2.871458,589922,589922,4306,1995
20,"""Money Train""","""Crime""",2.871458,589922,589922,4306,1995
20,"""Money Train""","""Drama""",2.871458,589922,589922,4306,1995
20,"""Money Train""","""Thriller""",2.871458,589922,589922,4306,1995


Considering the purpose of the project... decide if we need to delete the movies without values

Rita's opinion: se não temos informação sobre o ano/género/rating, e essas variáveis forem importantes para o modelo, podemos dar drop dessas rows por mim

In [ ]:
#drop a linhas com null values
#df_clean = df_exploded.drop_nulls()

### **2.2. Exploratory data analysis**

### **2.3. Feature Engineering**

Criar novas variáveis:
-	n_ratings → nº de avaliações por filme
-	avg_rating → média de rating por filme
-	genre_count → nº de géneros atribuídos
-	popularity_score = log(1 + nº de ratings) × média (para normalizar popularidade)

## **3. Models**

Objetivo: medir o peso de cada fator sobre a média de avaliação.

Modelos possíveis:
- Regressão Linear (OLS) → prever avg_rating com base em n_ratings, genre_count e dummies dos géneros principais.
- Árvore de decisão / Random Forest → avaliar a importância relativa das variáveis.

Métricas: R², RMSE, importância de features.


## **4. Visualization and Anaysis**

Gráficos:
- Scatter n_ratings vs avg_rating (com tendência)
- Boxplots de avg_rating por género
- Importância das features no modelo	

Insights:
- Géneros com maiores médias
- Géneros mais populares (maior número de users)
- Relação entre popularidade e qualidade percebida


## **5. Escalability and big_data**

Executar as queries principais em DuckDB e exportar para Parquet.
- Demonstrar (mesmo que parcialmente) a execução de consultas analíticas escaláveis:

SELECT genre, COUNT(*) AS n_filmes, AVG(rating) AS avg_rating

FROM ratings

JOIN movies USING(movieId)

GROUP BY genre;

- Mostrar que a mesma lógica pode ser usada em Athena / BigQuery com datasets maiores (ex: MovieLens 25M).

## **5. Conclusions**

	•	Identificar que fatores explicam melhor as boas avaliações.
	•	Propor extensão:
	•	Modelo preditivo por utilizador (colaborativo)
	•	Integração com tags para enriquecer o conteúdo
